In [4]:
import json
import requests
from requests.auth import HTTPBasicAuth
import pandas as pd
import time
from datetime import datetime
import geopandas as gpd
from shapely.geometry import Polygon


PLANET_API_KEY='PLAK93320cfc5cdc4acd81c3df746739143a'

orders_url = 'https://api.planet.com/compute/ops/orders/v2'

# set up requests to work with api
auth = HTTPBasicAuth(PLANET_API_KEY, '')
headers = {'content-type': 'application/json'}

course_code = "01A"
year = "2025"

#Directory paths
bDir = 'C:\\Users\\msmurphy\\Documents\\WinterTurf Planet\\Sensor Courses\\'
iDir = bDir + 'geojson'
oDir = bDir + 'images'

print(datetime.now())


2025-12-05 11:59:59.420448


In [5]:
#some json functions

def open_geojson(file_path):
    with open(file_path) as f:
#         gj = geojson.load(f)
        gj = json.load(f)
        
    return gj

def get_geojson_geometry(geojson):
    geometry = [i['geometry'] for i in geojson['features']]
    
    return geometry

def plot_wrapper(gdf, ax, color, linewidth=1.5):
    '''Convenience function for overlaying spatial data on the same plot'''
    gdf.plot(
        facecolor="none",
        edgecolor=color,
        linewidth=linewidth,
        ax = ax   
    )

# define helpful functions for submitting, polling, and downloading an order
def place_order(request, auth):
    response = requests.post(orders_url, data=json.dumps(request), auth=auth, headers=headers)
    print(response)
    
    if not response.ok:
        raise Exception(response.content)

    order_id = response.json()['id']
    print(order_id)
    order_url = orders_url + '/' + order_id
    return order_url

# print(datetime.now())

# Function to retrieve all features
def get_all_features(search_params, auth):
    features = []
    offset = 0
    while True:
        search_params['offset'] = offset
        response = requests.get(orders_url, headers=headers, auth=auth, params=search_params)
        response.raise_for_status()  # Raise an error for bad requests
        data = response.json()
        # print(data)
        features.extend(data['features'])
        if len(data['features']) < 250:
            break
        offset += 250
    return features

print(datetime.now())

2025-12-05 12:00:01.481284


In [6]:

feature_class_name = "Course_"+course_code+"_Outline"

start_date = year+'-01-01'
end_date = year+'-12-31'

geojson_file = iDir+"\\"+feature_class_name+".geojson"

# Read the GeoJSON file into a GeoDataFrame for the aoi
gdf_aoi = gpd.read_file(geojson_file)

#Create the geometry for the Planet query
boundary_geojson = open_geojson(geojson_file)

bounds = get_geojson_geometry(boundary_geojson)

geojson_geometry = bounds[0]

#Set up filters
geometry_filter = {
"type": "GeometryFilter",
"field_name": "geometry",
"config": geojson_geometry
}

instrument_filter = {
"type":"StringInFilter",
"field_name":"instrument",
"config":[
    # "PSB.SD"
    "PS2.SD"
    # "PS2"
]
}

date_range_filter = {
"type": "DateRangeFilter",
"field_name": "acquired",
"config": {
    "gte": start_date+"T00:00:00.000Z",
    "lte": end_date+"T00:00:00.000Z"
}
}

# orthoanalytic images that have 4 bands and are corrected for surface reflectance
asset_filter = {
"type": "AssetFilter",
"config": ["ortho_analytic_4b_sr"]
}

# combine filters including clear filters
was_combined_filter = {
"type": "AndFilter",
"config": [geometry_filter, date_range_filter, asset_filter]
}

item_type = "PSScene"

# API request object
search_request = {
"item_types": [item_type], 
# "limit":250,
# "offset": 0,
"filter": was_combined_filter
}

# # fire off the POST request
search_result = \
requests.post(
    'https://api.planet.com/data/v1/quick-search?_sort=acquired asc&_page_size=250',
    auth=HTTPBasicAuth(PLANET_API_KEY, ''),
    json=search_request)


search_result.raise_for_status()
payload = search_result.json()

# Collect results
all_features = []
all_ids = []

# Add first page features
features = payload.get("features", []) or []
all_features.extend(features)
all_ids.extend([f.get("id") for f in features])

# Get the next link (if any)
links = payload.get("_links", {}) or {}
next_url = links.get("_next")

SLEEP_SECONDS = 0.25
TIMEOUT = 60.0

# --- Follow pagination chain (GET) ---
while next_url is not None:
    if SLEEP_SECONDS:
        time.sleep(SLEEP_SECONDS)

    nxt = requests.get(next_url, auth=auth, timeout=TIMEOUT)

    # Simple retry on rate limit (HTTP 429)
    if nxt.status_code == 429:
        time.sleep(2.0)
        nxt = requests.get(next_url, auth=auth, timeout=TIMEOUT)

    nxt.raise_for_status()
    payload = nxt.json()

    features = payload.get("features", []) or []
    all_features.extend(features)
    all_ids.extend([f.get("id") for f in features])

    links = payload.get("_links", {}) or {}
    next_url = links.get("_next")

print(f"Fetched {len(all_features)} features")

#Create a dataframe out of the json response to the search
df_tiles = pd.json_normalize(all_features, max_level=1)

# rename properties column, dropping the properties label
prop_cols = {col: col.split('.')[1] for col in df_tiles.columns if 'properties' in col}
df_tiles.rename(columns=prop_cols, inplace=True)

geoms = [Polygon(geo[0]) for geo in df_tiles['geometry.coordinates']]
gdf_tiles = gpd.GeoDataFrame(df_tiles, geometry = geoms, crs=4326)

gdf_tiles_contain_aoi = gpd.sjoin(gdf_tiles, gdf_aoi,  how='inner', predicate='contains' )

gdf_tiles_contain_aoi['date_acquired'] = pd.to_datetime(gdf_tiles_contain_aoi['acquired'].str[:10])

# #sort by clear percentage
# try:
#     # df_tiles_sorted = gdf_tiles_contain_aoi.sort_values(by='clear_percent', ascending=False)
#     df_tiles_sorted = gdf_tiles_contain_aoi.sort_values(by='clear_confidence_percent', ascending=False)
#     # Drop duplicates and keep the first occurrence of each date (highest clear percentage)
#     df_tiles_unique = df_tiles_sorted.drop_duplicates(subset='date_acquired', keep='first')
# except:
#     print("Couldn't sort by clear confidence percentage")
#     df_tiles_unique = gdf_tiles_contain_aoi.drop_duplicates(subset='date_acquired', keep='first')

gdf_tiles.to_csv("C:\\Users\\msmurphy\\Documents\\WinterTurf Planet\\Sensor Courses\\Output_Index_All_Test.csv")
gdf_tiles_contain_aoi.to_csv("C:\\Users\\msmurphy\\Documents\\WinterTurf Planet\\Sensor Courses\\Output_Index_Contain_AOI_Test.csv")

Fetched 560 features
